In [ ]:
import cv2
import numpy as np

# Load the background image
background_img = cv2.imread(r"C:\Users\saru5\OneDrive\Pictures\background_image.jpg")

# Open the webcam
cap = cv2.VideoCapture(0)  # Use '0' for the default camera

# Set a smaller resolution for faster processing
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)  # Set the width to 320 pixels
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)  # Set the height to 240 pixels

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video camera.")
else:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Could not read frame.")
            break

        # Resize the background to match the frame dimensions
        background_img_resized = cv2.resize(background_img, (frame.shape[1], frame.shape[0]))

        # Convert the frame to HSV color space for better green detection
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define the green screen color range in HSV
        lower_green = np.array([35, 40, 40])   # Lower bound for green
        upper_green = np.array([85, 255, 255]) # Upper bound for green

        # Create a mask to detect green areas
        mask = cv2.inRange(hsv_frame, lower_green, upper_green)
        mask_inv = cv2.bitwise_not(mask)  # Invert mask for non-green areas

        # Extract the subject from the frame using the inverted mask
        subject = cv2.bitwise_and(frame, frame, mask=mask_inv)

        # Extract the background area from the new background image where green was detected
        background_area = cv2.bitwise_and(background_img_resized, background_img_resized, mask=mask)

        # Combine the subject and background area
        combined_frame = cv2.add(subject, background_area)

        # Display the resulting frame
        cv2.imshow("Live Green Screen Replacement", combined_frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()
